# Procesamiento y Análisis de Archivos Multimedia


## Contenidos:
* [Librerías](#Librerías)
* [Descarga](#Descarga)
* [Procesamiento](#Procesamiento)
* [Análisis](#Análisis)
* [Conclusiones](#Conclusiones)
* [Referencias](#Referencias)

### Librerías

In [1]:
## librerias principales
import yt_dlp # libreria para descargar videos de youtube
import whisper # libreria para usar modelo IA de Speech to Text
import openai # libreria para usar LLM de OpenAI


## librerias secundarias
import os # libreria para manejar archivos y directorios
from typing import List # libreria para manejar listas
import time  # libreria para manejar tiempo

In [2]:
openai.api_key = "sk-XXXXXXXXXXXXXXXX"

Para este notebook contamos con un par de rutas pre-definidas, una para la carpeta de descargas, una carpeta para la transcripción y otra para la carpeta de procesamiento (resumen del archivo multimedia).

<div class="alert alert-block alert-info">
<b>Multimedia Path:</b> "data/videos/".
<br>
<b>Transcriptions Path:</b> "data/transcripts/".
<br>
<b>Processing Path:</b> "data/summaries/".
</div>

 Además, se importan las librerías necesarias para el procesamiento y análisis de archivos multimedia.

### Descarga
Para descargar el contenido desde Youtube localmente tenemos varias opciones, en este momento optare por descargarlo usando la libreria de yt_dlp. Escogí esta libreria pues es una de las mas populares.
Un requisito importante es tener instalado [ffmpeg]("https://github.com/FFmpeg/FFmpeg"), para poder descargar y procesar los archivos multimedia.

In [3]:
# funcion para descargar multimedia de redes sociales usando yt_dlp
def download_audio(url: str) -> None:
    """Download audio from youtube video
    Args:
        url (str): url of the video
    """
    path = "data/videos/" 
    ydl_opts = {
        ## Para descargar solo audio
        'format': 'bestaudio/best', # download best audio
        'outtmpl': path + '%(title)s.mp3', 

        ## Para descargar video + audio
        #'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best', # download best video and audio
        #'outtmpl': path + '%(title)s.%(ext)s', 
    }


    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    file_name = ydl.prepare_filename(ydl.extract_info(url, download=False))
    file_name = file_name.split("data\\videos\\")[1]
    print("Downloaded audio '{}' successfully".format(file_name))

    return file_name # regresa el nombre del archivo descargado

In [4]:
# Usando la funcion para descargar el audio de un video de youtube

url = "https://www.youtube.com/watch?v=QGdbs2S2YHk"

archivo_multimedia = download_audio(url) # guardamos el nombre de nuestro archivo



[youtube] Extracting URL: https://www.youtube.com/watch?v=QGdbs2S2YHk
[youtube] QGdbs2S2YHk: Downloading webpage
[youtube] QGdbs2S2YHk: Downloading ios player API JSON
[youtube] QGdbs2S2YHk: Downloading android player API JSON


[youtube] QGdbs2S2YHk: Downloading player 0af6e327
[youtube] QGdbs2S2YHk: Downloading m3u8 information
[info] QGdbs2S2YHk: Downloading 1 format(s): 251
[download] Destination: data\videos\How to Learn (a lot).mp3
[download] 100% of   13.97MiB in 00:00:01 at 8.34MiB/s     
[youtube] Extracting URL: https://www.youtube.com/watch?v=QGdbs2S2YHk
[youtube] QGdbs2S2YHk: Downloading webpage
[youtube] QGdbs2S2YHk: Downloading ios player API JSON
[youtube] QGdbs2S2YHk: Downloading android player API JSON


[youtube] QGdbs2S2YHk: Downloading m3u8 information
Downloaded audio 'How to Learn (a lot).mp3' successfully


### Procesamiento
Para el procesamiento crearemos un par de funciones que nos permitan extraer el audio del video y transcribirlo a texto. Para la transcripción usaremos el modelo open-source de OpenAI, llamado [Whisper]("https://github.com/openai/whisper").

In [5]:
# funcion para cargar el modelo Whisper
def load_whisper(model_name: str = "medium"):
    """Load whisper model
    Args:
        model_name (str): name of the model
    """
    return whisper.load_model(model_name)

# funcion para transcribir el audio a texto
def audio2text(file, model, language: str="English") -> str:
    """Transcribe audio to text using whisper model"""
    return model.transcribe(file, fp16=False, language=language)

# funcion para guardar la transcripcion en un archivo de texto
def save_text(text: str, file: str) -> None:
    """Save text to file"""
    try: 
        with open(file, "w") as f:
            f.write(text)
    except Exception as e:
        print("Error saving text to file: {}".format(e))

# funcion para obtener el texto de un archivo de audio
def get_text(file_name: str, model):
    input_file = "data/videos/" + file_name
    trans_path = "data/transcripts/" # path to transcripts
    # now lets transcribe the audio
    res = audio2text(input_file, model)
    save_text(res['text'], trans_path + file_name.split(".")[0] + "_transcript.txt")
    print("Transcript saved successfully")

In [6]:
# primero cargamos el modelo Whisper
model = load_whisper("small") # dependiendo de la GPU donde se ejecute el codigo, se puede cambiar el tamaño del modelo

# ahora transcribimos el audio a texto
get_text(archivo_multimedia, model)

Transcript saved successfully


### Análisis

En esta ocasión, se realizará un análisis de sobre el texto transcribido. Para ello, se utilizará la el modelo de lenguaje GPT de OpenAI, al cual en esta ocasión sera consultado mediante el API.</br></br>
Para ello creamos un prompt con el cual solicitamos al modelo que nos genere un resumen del texto transcribido.

In [7]:
# Recibir respuesta del modelo GPT de OpenAI

## Configuracion del prompt para el modelo
prompt = """Podrias darme un resumen a detalle de la transcripcion del siguiente video titulado: '{}'
            el video es de James Briggs, un reconocido ingeniero de IA que comparte contenido educativo en su canal de Youtube""".format(archivo_multimedia.split(".")[0])

## Cargar la transcripcion del archivo de texto
with open("data/transcripts/{}_transcript.txt".format(archivo_multimedia.split(".")[0]), "r") as file:
    transcript = file.read()

## Establecer la configuracion del modelo
delay_time = 0.01
max_tokens = 2000
start_time = time.time()

## Seleccionamos el modelo GPT a usar
GPT_Model = "gpt-4-turbo"

def GPT_Summarize(prompt, transcript, max_tokens, GPT_Model):
    # GPT Chabot API Request
    response = openai.ChatCompletion.create(
        model = GPT_Model,
        messages = [
            {'role': 'system', 'content': 'You are a helpful NLP assistant.'}, # especificamos el rol del modelo
            {'role': 'user', 'content': prompt}, # especificamos el prompt
            {'role': 'user', 'content': f'Transcript: {transcript}'}, # especificamos el contenido del prompt
        ],
        max_tokens = max_tokens,
        temperature = 0.7,
    )
    print("Summary generated successfully")
    return response.choices[0].message['content']

def Save_Summary(summary, file_name):
    with open("data/summaries/{}_summary-GPT.txt".format(file_name.split(".")[0]), "w") as file:
        file.write(summary)
    print("Summary saved successfully")

In [8]:
# Ejecutar el modelo GPT para el Resumen del video Multimedia
summary = GPT_Summarize(prompt, transcript, max_tokens, GPT_Model)
Save_Summary(summary, archivo_multimedia.split(".")[0]) # Guardar el resumen en un archivo de texto

Summary generated successfully
Summary saved successfully


### Conclusiones

Con este notebook se pretende mostrar un ejemplo de como se puede procesar y analizar archivos multimedia de manera automatizada. En este caso, se descargó un video de Youtube, se extrajo el audio, se transcribió a texto y se analizó el contenido del texto transcribido usando 2 diferentes modelos de IA en python.

### Referencias

<div align="center">

| Versión | Fecha     | Autor                 |
|:-------:|:---------:|:---------------------:|
| 1.0     | 18/04/2024| [Carlos Arreola](https://github.com/CharlieArreola) |

</div>
